# 학습모델

In [1]:
""" 
Created on Wed Jan 23 2019
 
MTS Program that graphs difference in mahalanobis distances, as well as an MTS 
 
@author: Taehee 
""" 

from __future__ import print_function
import os
import numpy as np 
import matplotlib as plt 
import copy 
import math
import csv
import pandas as pd
from sklearn import preprocessing

In [2]:
# csv 파일을 저장할 위치 확인
print (os.getcwd())

C:\Users\mando\Desktop\ssd


In [3]:
# 파일 읽기 ->> DB에서 다운받은 CSV를 바로 읽어 OK, NG 구분할 수 있는 기능 추가 필요
data=pd.read_csv('SAM2(11.01~11.27).CSV',encoding='utf-8' )
print(data.columns)
# 원하는 컬럼만 가져옴
factor = ['축력 하중(kgf)','컬링 하중(kgf)','토크상한 누적시간','RPM 편차(rpm)','높이측정 - 완료위치(mm)']
OKdata = data[factor]
OKdata


Index(['날짜', '시간', 'W/C', '차종', '시리얼 No.', '판정값(MD)', '판정', '축력 하중(kgf)',
       '컬링 하중(kgf)', '토크상한 누적시간', 'RPM 편차(rpm)', '높이측정(mm)', '완료위치(mm)',
       '높이측정 - 완료위치(mm)', '단차 측정(mm)', 'MD Spec(1차)', 'MD Spec(2차)', '조치 판정',
       '롤컬링 수집 상태', 'MD 수집 상태'],
      dtype='object')


,축력 하중(kgf),컬링 하중(kgf),토크상한 누적시간,RPM 편차(rpm),높이측정 - 완료위치(mm)
0,1728.0,2253.0,0.027,47.0,28.2
1,1715.0,2190.0,0.026,47.0,28.2
2,1715.0,2316.0,0.029,48.0,28.2
3,1712.0,2247.0,0.024,46.0,28.2
4,1699.0,2364.0,0.025,45.0,28.1
5,1741.0,2187.0,0.028,48.0,28.2
6,1715.0,2234.0,0.025,48.0,28.2
7,1722.0,2225.0,0.027,48.0,28.2
8,1725.0,2196.0,0.025,48.0,28.3
9,1725.0,2304.0,0.025,47.0,28.1


In [4]:
# n, k, factor별 mean, std 확인
print(OKdata.shape)
n = OKdata.shape[0]
k = OKdata.shape[1]
print("제품 수(n):",n,",","factor 수(k):",k)
OKdata_factor_mean = OKdata[factor].mean(axis=0)
OKdata_factor_mean
OKdata_factor_std = OKdata[factor].std(axis=0)
OKdata_factor_std


(57158, 5)
제품 수(n): 57158 , factor 수(k): 5


축력 하중(kgf)         23.682488
컬링 하중(kgf)         70.763083
토크상한 누적시간           0.003182
RPM 편차(rpm)        11.353741
높이측정 - 완료위치(mm)     0.102581
dtype: float64

In [5]:
# 데이터 정규화(Z)
scaler = preprocessing.StandardScaler()
scaled_df_ok = scaler.fit_transform(OKdata)

print(scaled_df_ok)

# 상관계수(R) 구하기
pd_scaled_df_ok = pd.DataFrame(scaled_df_ok)
OKdata_corr = pd_scaled_df_ok.corr()
OKdata_corr
OKdata_corr.shape

# 상관계수의 역함수(R^-1)
cor =np.linalg.inv(OKdata_corr)
cor
cor.shape

[[ 0.49625016  0.2734307   0.09396411 -0.01460972  0.43948242]
 [-0.05268346 -0.61687182 -0.22031996 -0.01460972  0.43948242]
 [-0.05268346  1.16373323  0.72253225  0.07346774  0.43948242]
 ...
 [ 0.62292715  2.19535361  0.09396411 -0.01460972  0.43948242]
 [ 0.62292715  1.84205896  0.72253225  0.07346774 -0.53536764]
 [ 0.49625016 -0.39076324  3.2368048  -0.10268719  0.43948242]]


(5, 5)

# 검사모델 

In [1]:
# pymssql패키지 imoprt
import pymssql

In [2]:
# mssql 접속
conn = pymssql.connect(server='SQLEXPRESS', database='ROLLCURLING_MONITORING', charset='utf8')
#connection에서 cursor 생 (cursor는 fetch 동작 관리)
cursor = conn.cursor()
# OS와 맞는 DB설치할 것


InterfaceError: Connection to the database failed for an unknown reason.

아래 SQL 문만 바꿔주면 된다

In [ ]:
cursor.execute('SELECT VALUE01, VALUE02, VALUE03, VALUE04, VALUE07 FROM QMS_MONITORING_DATA WHERE VALUE01 IS NOT NULL ;')
    # VALUE01 : 축력하중(kgf)
    # VALUE02 : 컬링하중(kgf)
    # VALUE03 : 토크상한 누적시간
    # VALUE04 : RPM 편차(rpm)
    # VALUE05 : 높이측정(mm) (사용x)
    # VALUE06 : 완료위치(mm) (사용x)
    # VALUE07 : 높이측정-완료위치(mm)
    # VALUE08 : 단차 측정(mm) (SAM2라인 사용X)

In [342]:
for i in range(5):
    row_array = []
    z_array = []

    row = cursor.fetchone()
    print("raw data:", row)
    row_array = np.array(row)
    #print(row_array.shape)

    for i in range(k):
        #print("raw data:",float(row_array[i]))
        #print("mean:",OKdata_factor_mean[i])
        #print("std:",OKdata_factor_std[i])
        data_minus_mean = float(row_array[i])-OKdata_factor_mean[i]
        z = data_minus_mean/OKdata_factor_std[i]
        #print("z:",z)
        z_array.append(z)

    print("z:", z_array)
    z_transpose = np.transpose(z_array)
    #print(z_transpose)
    MD_1 = np.dot(z_array,cor)
    MD = np.dot(MD_1, z_transpose)
    D = np.sqrt(MD)

    print("MD:",MD,",","D:",D)

array([], dtype=float64)

In [ ]:
curs.close() 